In [1]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import time
import re
from google.colab import drive
from tqdm import tqdm
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
researcher=pd.read_csv('/content/drive/MyDrive/academic_tree/data/researcher.csv')
gender=pd.read_csv('/content/drive/MyDrive/academic_tree/data/first_name_gender.csv')
race=pd.read_csv('/content/drive/MyDrive/academic_tree/data/full_name_race.csv')

In [ ]:
race

,FullName,race
0,STEPHEN V DAVID,White
1,BENJAMIN Y HAYDEN,White
2,JACK L GALLANT,White
3,BENJAMIN WILLMORE,White
4,KENDRICK NORRIS KAY,White
...,...,...
764095,ANNY REYES,Hispanic
764096,JOHN A LEE,Asian
764097,DAVID L SMISEK,White
764098,JOHN F FARRAR,White


In [3]:
researcher=researcher.merge(gender,on=['FirstName'],how='left')
researcher["FullName"] = (
    researcher["FirstName"].astype(str) +  # 确保所有列是字符串类型
    " " +
    researcher["MiddleName"].fillna("").astype(str) +  # 处理 None/NaN
    " " +
    researcher["LastName"].astype(str)
).str.strip()  # 去除首尾多余空格
race['FullName'] = race['FullName'].str.upper()
researcher['FullName'] = researcher['FullName'].str.upper()
researcher=researcher.merge(race,on=['FullName'],how='left')

In [4]:
researcher.gender.value_counts()

,count
gender,
man,374434
woman,264210
unknown,135899


In [5]:
researcher.race.value_counts()

,count
race,
White,318337
Asian,44639
Hispanic,38070
Black,10294


In [6]:
researcher

,PID,FirstName,MiddleName,LastName,Institution,InstitutionMAGID,ResearchArea,ORCID,MAGAuthorID,gender,FullName,race
0,1,STEPHEN,V,DAVID,Oregon Health and Science University,165690674.0,"neuro,csd,bme",0000-0003-4135-3104,2.171828e+09,man,STEPHEN V DAVID,White
1,2,BENJAMIN,Y,HAYDEN,"University of Minnesota, Twin Cities",130238516.0,neuro,0000-0002-7678-4281,2.082173e+09,man,BENJAMIN Y HAYDEN,White
2,3,JACK,L,GALLANT,"University of California, Berkeley",95457486.0,"neuro,psych",NaN,2.110166e+09,man,JACK L GALLANT,White
3,4,BENJAMIN,NaN,WILLMORE,University of Oxford,40120149.0,neuro,NaN,2.183264e+09,man,BENJAMIN WILLMORE,NaN
4,5,KENDRICK,NORRIS,KAY,"University of Minnesota, Twin Cities",130238516.0,neuro,NaN,2.156587e+09,unknown,KENDRICK NORRIS KAY,White
...,...,...,...,...,...,...,...,...,...,...,...,...
774741,839677,ANNY,NaN,REYES,"University of California, San Diego",36258959.0,neuro,NaN,2.554457e+09,woman,ANNY REYES,NaN
774742,839678,JOHN,A,LEE,University of Sheffield,91136226.0,evol,NaN,2.123316e+09,man,JOHN A LEE,Asian
774743,839679,DAVID,L,SMISEK,NaN,NaN,chemistry,NaN,2.547457e+09,man,DAVID L SMISEK,White
774744,839680,JOHN,F,FARRAR,University of Wales,97429440.0,evol,NaN,2.153002e+09,man,JOHN F FARRAR,White


In [7]:
# 先填充NA为空字符串（可选）
researcher['ResearchArea'] = researcher['ResearchArea'].fillna('')
# 使用str访问器进行向量化操作
researcher['ResearchArea'] = researcher['ResearchArea'].str.split(',').str[0].str.strip()
# 将空字符串转回None（可选）
researcher['ResearchArea'] = researcher['ResearchArea'].replace('', None)

In [8]:
researcher['ResearchArea'].value_counts()

,count
ResearchArea,
neuro,135222
chemistry,102156
etree,56736
educ,56544
physics,48920
...,...
ecotox,2
nutrition,2
bone,1


In [9]:
researcher.drop(['ORCID','FullName'],axis=1,inplace=True)

In [10]:
researcher

,PID,FirstName,MiddleName,LastName,Institution,InstitutionMAGID,ResearchArea,MAGAuthorID,gender,race
0,1,STEPHEN,V,DAVID,Oregon Health and Science University,165690674.0,neuro,2.171828e+09,man,White
1,2,BENJAMIN,Y,HAYDEN,"University of Minnesota, Twin Cities",130238516.0,neuro,2.082173e+09,man,White
2,3,JACK,L,GALLANT,"University of California, Berkeley",95457486.0,neuro,2.110166e+09,man,White
3,4,BENJAMIN,NaN,WILLMORE,University of Oxford,40120149.0,neuro,2.183264e+09,man,NaN
4,5,KENDRICK,NORRIS,KAY,"University of Minnesota, Twin Cities",130238516.0,neuro,2.156587e+09,unknown,White
...,...,...,...,...,...,...,...,...,...,...
774741,839677,ANNY,NaN,REYES,"University of California, San Diego",36258959.0,neuro,2.554457e+09,woman,NaN
774742,839678,JOHN,A,LEE,University of Sheffield,91136226.0,evol,2.123316e+09,man,Asian
774743,839679,DAVID,L,SMISEK,NaN,NaN,chemistry,2.547457e+09,man,White
774744,839680,JOHN,F,FARRAR,University of Wales,97429440.0,evol,2.153002e+09,man,White


In [11]:
researcher.to_csv('/content/drive/MyDrive/academic_tree/data/researcher.csv',index=False)